In [0]:

/*
create table hive_metastore.default.cm_5g_cell_records_bk_20241118
select * from hive_metastore.default.cm_5g_cell_records;

select count(*) from hive_metastore.default.cm_5g_cell_records_bk_20241118;  -- 116517358
select count(*) from hive_metastore.default.cm_5g_cell_records;   -- 116517358

select * 
-- delete
from hive_metastore.default.cm_5g_cell_records where Date >= '2024-10-08' and Date <= '2024-10-20';

*/



delete from hive_metastore.default.cm_5g_cell_records where date=CURRENT_DATE()-1;



INSERT INTO hive_metastore.default.cm_5g_cell_records
SELECT current_date()-1 as Date,gnb, du_raw_id, cellidentity, max(frequency_band_indicator) frequency_band_indicator
FROM read_files(
  's3://apacheupsertdata/cm_data_config/cm_5g_cell_records_{{date}}.csv',
  format => 'csv',
  header => true,
  mode => 'FAILFAST')
  where frequency_band_indicator is not null
  GROUP BY ALL
  ;

/*
INSERT INTO hive_metastore.default.cm_5g_cell_records
SELECT current_date()-1 as Date,gnb, du_raw_id, cellidentity, frequency_band_indicator
FROM read_files(
  's3://apacheupsertdata/cm_data_config/cm_5g_cell_records_{{date}}.csv',
  format => 'csv',
  header => true,
  mode => 'FAILFAST')
  where frequency_band_indicator is not null
  ;
*/

TRUNCATE TABLE `hive_metastore`.`default`.`vz_conf_gnode_du_gcell_silver_tmp`;

/*
insert into `hive_metastore`.`default`.`vz_conf_gnode_du_gcell_silver_tmp`
(GNODEB_ID,
DU_ID,
GCELL_ID,
MARKET_NAME,
NR_TYPE,
SUB_MARKET_ID,
SUB_MARKET_NAME,
CC_ID,
NR_TYPE_per_gNB,
FREQUENCY_BAND_INDICATOR,
DU_NAME,
NETWORK_NAME
)
select distinct concat('VZ_',left(regexp_replace(a.NE_NAME, '[^0-9]', ''), 7)) AS GNODEB_ID,
concat('VZ_',left(regexp_replace(a.NE_NAME, '[^0-9]', ''),11)) AS DU_ID,
cast(replace(a.LOCATION, 'CellIdentity', '') as bigint) AS GCELL_ID,
b.Region AS MARKET_NAME,
case 
  when right(left(left(regexp_replace(a.NE_NAME, '[^0-9]', ''),11), 4), 1)=0 then 'FR2'
  when right(left(left(regexp_replace(a.NE_NAME, '[^0-9]', ''),11), 4), 1)=9 and right(left(left(regexp_replace(a.NE_NAME, '[^0-9]', ''),11), 8), 1)=2 then 'FR1/LS3'
  when right(left(left(regexp_replace(a.NE_NAME, '[^0-9]', ''),11), 4), 1)=9 and right(left(left(regexp_replace(a.NE_NAME, '[^0-9]', ''),11), 8), 1)=3 then 'FR1/LS6' 
  else NULL
end AS NR_TYPE,
cast(replace(a.SITE_ID, 'N/A', '') as bigint) AS SUB_MARKET_ID,
b.SUB_MARKET_NAME,
case 
  when right(left(left(regexp_replace(a.NE_NAME, '[^0-9]', ''),11), 4), 1)=9 and right(left(left(regexp_replace(a.NE_NAME, '[^0-9]', ''),11), 8), 1)=3 and MOD(replace(a.Location, 'CellIdentity',''), 2) = 0 then 'CC1' 
  when right(left(left(regexp_replace(a.NE_NAME, '[^0-9]', ''),11), 4), 1)=9 and right(left(left(regexp_replace(a.NE_NAME, '[^0-9]', ''),11), 8), 1)=3 and MOD(replace(a.Location, 'CellIdentity',''), 2) <> 0 then 'CC2'
  else NULL 
end AS CC_ID,
case 
  when right(left(left(regexp_replace(a.NE_NAME, '[^0-9]', ''),7), 4), 1)=0 then 'FR2'
  when right(left(left(regexp_replace(a.NE_NAME, '[^0-9]', ''),7), 4), 1)=9 then 'FR1'
  else NULL
end AS NR_TYPE_per_gNB,
c.FREQUENCY_BAND_IND,
null as DU_NAME,
null as NETWORK_NAME
from apachedataingestion.nr.cellunavailabletime_nr_hr_silver a
left join hive_metastore.default.vz_conf_market b on cast(replace(a.SITE_ID, 'N/A', '') as bigint) = b.SUB_MARKET_ID
left join (
  select a.*
  from hive_metastore.default.cm_5g_cell_records a
  inner join (select Max(Date) As Date,GNODEB_ID, DU_ID, GCELL_ID
              from hive_metastore.default.cm_5g_cell_records where Date>=(CURRENT_DATE - INTERVAL 4 DAY)
              GROUP BY GNODEB_ID, DU_ID,GCELL_ID 
  ) b on a.GNODEB_ID=b.GNODEB_ID and a.DU_ID=b.DU_ID and a.GCELL_ID=b.GCELL_ID and a.date=b.date
  where a.Date>=(CURRENT_DATE - INTERVAL 4 DAY)
)c on cast(trim(left(regexp_replace(a.NE_NAME, '[^0-9]', ''), 7)) as bigint) = cast(trim(c.GNODEB_ID) as bigint) and cast(trim(left(regexp_replace(a.NE_NAME, '[^0-9]', ''),11)) as bigint) = cast(trim(c.DU_ID) as bigint) and cast(replace(a.LOCATION, 'CellIdentity', '') as bigint) = c.GCELL_ID
where a.Daily>=(CURRENT_DATE - INTERVAL 4 DAY);
*/



insert into `hive_metastore`.`default`.`vz_conf_gnode_du_gcell_silver_tmp`
(GNODEB_ID,
DU_ID,
GCELL_ID,
MARKET_NAME,
NR_TYPE,
SUB_MARKET_ID,
SUB_MARKET_NAME,
CC_ID,
NR_TYPE_per_gNB,
FREQUENCY_BAND_INDICATOR,
DU_NAME,
NETWORK_NAME
)
select distinct concat('VZ_',left(regexp_replace(a.NE_NAME, '[^0-9]', ''), 7)) AS GNODEB_ID,
concat('VZ_',left(regexp_replace(a.NE_NAME, '[^0-9]', ''),11)) AS DU_ID,
cast(replace(a.LOCATION, 'CellIdentity', '') as bigint) AS GCELL_ID,
b.Region AS MARKET_NAME,
case 
when right(left(left(regexp_replace(a.NE_NAME, '[^0-9]', ''),11), 4), 1)=0 then 'FR2'
when right(left(left(regexp_replace(a.NE_NAME, '[^0-9]', ''),11), 4), 1)=9 and right(left(left(regexp_replace(a.NE_NAME, '[^0-9]', ''),11), 8), 1)=2 then 'FR1/LS3'
when right(left(left(regexp_replace(a.NE_NAME, '[^0-9]', ''),11), 4), 1)=9 and right(left(left(regexp_replace(a.NE_NAME, '[^0-9]', ''),11), 8), 1)=3 then 'FR1/LS6' 
else NULL
end AS NR_TYPE,
cast(replace(a.SITE_ID, 'N/A', '') as bigint) AS SUB_MARKET_ID,
b.SUB_MARKET_NAME,
case 
when right(left(left(regexp_replace(a.NE_NAME, '[^0-9]', ''),11), 4), 1)=9 
and right(left(left(regexp_replace(a.NE_NAME, '[^0-9]', ''),11), 8), 1)=3 
and MOD(replace(a.Location, 'CellIdentity',''), 2) = 0 then 'CC1' 
when right(left(left(regexp_replace(a.NE_NAME, '[^0-9]', ''),11), 4), 1)=9 
and right(left(left(regexp_replace(a.NE_NAME, '[^0-9]', ''),11), 8), 1)=3 
and MOD(replace(a.Location, 'CellIdentity',''), 2) <> 0 then 'CC2'
else NULL 
end AS CC_ID,
case 
when right(left(left(regexp_replace(a.NE_NAME, '[^0-9]', ''),7), 4), 1)=0 then 'FR2'
when right(left(left(regexp_replace(a.NE_NAME, '[^0-9]', ''),7), 4), 1)=9 then 'FR1'
else NULL
end AS NR_TYPE_per_gNB,
c.FREQUENCY_BAND_IND,
null as DU_NAME,
null as NETWORK_NAME
from apachedataingestion.nr.cellunavailabletime_nr_hr_silver a
left join hive_metastore.default.vz_conf_market b on cast(replace(a.SITE_ID, 'N/A', '') as bigint) = b.SUB_MARKET_ID
left join (
  select a.*
  from hive_metastore.default.cm_5g_cell_records a
  inner join (select Max(Date) As Date,GNODEB_ID, DU_ID, GCELL_ID
from hive_metastore.default.cm_5g_cell_records
GROUP BY GNODEB_ID, DU_ID,GCELL_ID ) b on a.GNODEB_ID=b.GNODEB_ID and a.DU_ID=b.DU_ID and a.GCELL_ID=b.GCELL_ID and a.date=b.date
)c on cast(trim(left(regexp_replace(a.NE_NAME, '[^0-9]', ''), 7)) as bigint) = cast(trim(c.GNODEB_ID) as bigint) and cast(trim(left(regexp_replace(a.NE_NAME, '[^0-9]', ''),11)) as bigint) = cast(trim(c.DU_ID) as bigint) and cast(replace(a.LOCATION, 'CellIdentity', '') as bigint) = c.GCELL_ID
where a.Daily>=(CURRENT_DATE - INTERVAL 4 DAY);
--where a.Daily>='2024-04-14';

/*

 select *  from hive_metastore.default.cm_5g_cell_records where du_id='13201990016' and gcell_id=257 order by 1

select DU_ID, GCELL_ID,count(*),min(freqUENCY_BAND_INDICATOR),max(freqUENCY_BAND_INDICATOR)
 from hive_metastore.default.vz_conf_gnode_du_gcell_silver_tmp group by all having  count(*) > 1


select *
 from hive_metastore.default.vz_conf_gnode_du_gcell_silver_tmp where du_id='VZ_13201990016' and gcell_id=257

 select * from hive_metastore.default.vz_conf_gnode_du_gcell_silver
*/



MERGE INTO hive_metastore.default.vz_conf_gnode_du_gcell_silver AS a
USING hive_metastore.default.vz_conf_gnode_du_gcell_silver_tmp AS b
ON a.DU_ID = b.DU_ID AND a.GCELL_ID = b.GCELL_ID
WHEN MATCHED THEN
  UPDATE SET a.GNODEB_ID = b.GNODEB_ID,
             a.DU_ID = b.DU_ID,
             a.GCELL_ID = b.GCELL_ID,
             a.MARKET_NAME = b.MARKET_NAME,
             a.NR_TYPE = b.NR_TYPE,
             a.SUB_MARKET_ID = b.SUB_MARKET_ID,
             a.SUB_MARKET_NAME = b.SUB_MARKET_NAME,
             a.FREQUENCY_BAND_INDICATOR = b.FREQUENCY_BAND_INDICATOR,
             a.CC_ID = b.CC_ID,
             a.NR_TYPE_per_gNB = b.NR_TYPE_per_gNB,
             a.DU_NAME = b.DU_NAME,
             a.NETWORK_NAME = b.NETWORK_NAME;


             
insert into vz_conf_gnode_du_gcell_silver
(GNODEB_ID,
DU_ID,
GCELL_ID,
MARKET_NAME,
NR_TYPE,
SUB_MARKET_ID,
SUB_MARKET_NAME,
CC_ID,
NR_TYPE_per_gNB,
FREQUENCY_BAND_INDICATOR,
DU_NAME,
NETWORK_NAME
)
select 
a.GNODEB_ID,
a.DU_ID,
a.GCELL_ID,
a.MARKET_NAME,
a.NR_TYPE,
a.SUB_MARKET_ID,
a.SUB_MARKET_NAME,
a.CC_ID,
a.NR_TYPE_per_gNB,
a.FREQUENCY_BAND_INDICATOR,
a.DU_NAME,
a.NETWORK_NAME 
from vz_conf_gnode_du_gcell_silver_tmp a
left join vz_conf_gnode_du_gcell_silver b
on a.DU_ID = b.DU_ID AND a.GCELL_ID = b.GCELL_ID
WHERE b.DU_ID IS NULL;




/*

create table if not exists xxx_jwk_vz_conf_gnode_du_gcell_silver_bkp
select count(*) from hive_metastore.default.vz_conf_gnode_du_gcell_silver  -- 581606
select count(*) from hive_metastore.default.xxx_jwk_vz_conf_gnode_du_gcell_silver_bkp  -- 581848
*/



delete from hive_metastore.default.vz_conf_gnode_du_gcell_silver_4remove;

with aaa (
select GNODEB_ID,GCELL_ID,count(distinct DU_ID) from vz_conf_gnode_du_gcell_silver group by GNODEB_ID,GCELL_ID having count(distinct DU_ID)>1 -- 2 retruns
),
bbb (
select a.GNODEB_ID,a.DU_ID,a.GCELL_ID from hive_metastore.default.vz_conf_gnode_du_gcell_silver a inner join aaa b on a.GNODEB_ID=b.GNODEB_ID and a.GCELL_ID=b.GCELL_ID   -- comfare to conf
)
insert into hive_metastore.default.vz_conf_gnode_du_gcell_silver_4remove
select DU_ID,GCELL_ID from bbb where (DU_ID,GCELL_ID) not in (
                        select b.DU_ID,b.GCELL_ID
                        from apachedataingestion.nr.cellunavailabletime_nr_hr_silver a
                        inner join bbb b on concat('VZ_',left(REGEXP_REPLACE(NE_NAME, '[^0-9]', ''),11))=b.DU_ID and replace(a.LOCATION,'CellIdentity','')=b.gcell_id
                        where Daily  >=  date_format(date_add(now(), -1),"yyyy-MM-dd") 
                        group by b.DU_ID,b.GCELL_ID
);

delete 
from vz_conf_gnode_du_gcell_silver A where EXISTS (
        select DU_ID,GCELL_ID from hive_metastore.default.vz_conf_gnode_du_gcell_silver_4remove WHERE A.DU_ID = DU_ID AND A.GCELL_ID = GCELL_ID
 );
 

